In [1]:
# import libraries

import numpy as np
import pandas as pd
import datetime

# progress bar tool

from tqdm import tqdm

In [2]:
# merge the two transactions files

transactions_v1 = pd.read_csv('transactions.csv')
transactions_v2 = pd.read_csv('transactions_v2.csv')

# merge with reset index

transactions = transactions_v1.append(transactions_v2).reset_index(drop = True)

# sort the transactions by 'msno' and 'transaction_date'

transactions = transactions.sort_values(['msno', 'transaction_date'])

current_msno = ''

# create the following month-wise, year-wise columns along with columns for 'cancel_count', 'cutoff_count', 
# 'transaction_count', 'combo_days'

cols_2015 = ['2015{}'.format(str(i+1).zfill(2)) for i in range(12)]
cols_2016 = ['2016{}'.format(str(i+1).zfill(2)) for i in range(12)]
cols_2017 = ['2017{}'.format(str(i+1).zfill(2)) for i in range(3)]

cols = ['msno'] + cols_2015 + cols_2016 + cols_2017 + ['cancel_count', 'cutoff_count', 'transaction_count', 'combo_days']

cols_dict = {}

In [3]:
cols

['msno',
 '201501',
 '201502',
 '201503',
 '201504',
 '201505',
 '201506',
 '201507',
 '201508',
 '201509',
 '201510',
 '201511',
 '201512',
 '201601',
 '201602',
 '201603',
 '201604',
 '201605',
 '201606',
 '201607',
 '201608',
 '201609',
 '201610',
 '201611',
 '201612',
 '201701',
 '201702',
 '201703',
 'cancel_count',
 'cutoff_count',
 'transaction_count',
 'combo_days']

In [4]:
# create a reverse dictionary for the 'cols'

for i, v in enumerate(cols):
    cols_dict[v] = i
    
print(cols_dict)                                        # print 'cols_dict'

membership_records = [[] for i in range(len(cols))]

transaction_dates = []

membership_expire_dates = []

is_cancels = []

total_rows = len(transactions['msno'])                  # total rows in the 'transactions' dictionary

{'msno': 0, '201501': 1, '201502': 2, '201503': 3, '201504': 4, '201505': 5, '201506': 6, '201507': 7, '201508': 8, '201509': 9, '201510': 10, '201511': 11, '201512': 12, '201601': 13, '201602': 14, '201603': 15, '201604': 16, '201605': 17, '201606': 18, '201607': 19, '201608': 20, '201609': 21, '201610': 22, '201611': 23, '201612': 24, '201701': 25, '201702': 26, '201703': 27, 'cancel_count': 28, 'cutoff_count': 29, 'transaction_count': 30, 'combo_days': 31}


In [5]:
for i, row in tqdm(transactions.iterrows(), total = total_rows):
    
    msno = row['msno']                                                     # 'msno'
    transaction_date = row['transaction_date']                             # 'transaction_date'
    is_cancel = int(row['is_cancel'])                                      # 'is_cancel'
    membership_expire_date = row['membership_expire_date']                 # 'membership_expire_date'
    
    # current_msno is ''
    
    if(current_msno != msno or i == total_rows - 1):
        
        if(current_msno != ''):
            
            cutoff_count = 0
            cancel_count = sum(is_cancels)
            transaction_count = len(transaction_dates)
            
            # membership_record[-1] is number of consecutive current member
            # membership_record[-2] is transaction_count
            # membership_record[-3] is cutoff_count
            # membership_record[0] is msno
            
            membership_record = [-1 for j in range(len(cols))]
            
            left = int(transaction_dates[0])
            right = int(membership_expire_dates[0])
            
            for j in range(1, len(transaction_dates)):
                
                l = int(transaction_dates[j])
                r = int(membership_expire_dates[j])
                
                if(r < right):
                # update the expiry date, if unsubscribed
                
                    right = r
                    
                elif(l > right):
                # account without subscription for days after the expiry date
                # save the current membership status for that month
                
                    cutoff_count += 1
                    start = int(left / 100)                    # e.g. 20160223 / 100 = 201602
                    end = int(right / 100)                     # e.g. 20160223 / 100 = 201602
                    
                    for j in range(start, end + 1):
                        
                        if(j % 100 <= 12):
                            
                            if(str(j) in cols_dict):
                                
                                membership_record[cols_dict[str(j)]] = 1
                                
                    left = l
                    right = r
                    
                else:
                # renewal before expiration, extension of the membership
                # start date remains the same
                
                    right = r
                    
            # store the final membership status
            
            start = int(left / 100)                           # e.g. 20160223 / 100 = 201602
            end = int(right / 100)                            # e.g. 20160223 / 100 = 201602
            
            for j in range(start, end + 1):
                
                if(j % 100 <= 12):
                    
                    if(str(j) in cols_dict):
                        
                        membership_record[cols_dict[str(j)]] = 1
                        
            start_date = datetime.datetime.strptime(str(left), "%Y%m%d")
            end_date = datetime.datetime.strptime(str(right), "%Y%m%d")
            
            # update the membership features for the current user
            
            membership_record[0] = current_msno
            membership_record[-1] = (end_date - start_date).days
            membership_record[-2] = transaction_count
            membership_record[-3] = cutoff_count
            membership_record[-4] = cancel_count
            
            # append the features
            
            for j in range(len(cols)):
                
                membership_records[j].append(membership_record[j])
                
        transaction_dates = []
        membership_expire_dates = []
        is_cancels = []
        
    # append the column values
    
    transaction_dates.append(transaction_date)
    membership_expire_dates.append(membership_expire_date)
    is_cancels.append(is_cancel)
    
    # assign 'current_msno' as 'msno' for the next iteration
    current_msno = msno

100%|███████████████████████████████████████████████████████████████████| 22978755/22978755 [31:03<00:00, 12332.61it/s]


In [6]:
# create the output file

out = pd.DataFrame()

for i, v in enumerate(cols):
    out[v] = membership_records[i]
    
out.to_csv('membership_records.csv', index = False)